[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Shivanandroy/GameOfThrones3D/blob/master/Visualizing%20Game%20of%20Thrones%20using%20BERT.ipynb)

In [0]:
import pandas as pd
import numpy as np


!pip install plotly_express
!pip install bert-embedding
!pip install mxnet-cu100

import mxnet as mx

In [0]:
## 
book1 = "https://raw.githubusercontent.com/llSourcell/word_vectors_game_of_thrones-LIVE/master/data/got1.txt"
book2 = "https://raw.githubusercontent.com/llSourcell/word_vectors_game_of_thrones-LIVE/master/data/got2.txt"
book3 = "https://raw.githubusercontent.com/llSourcell/word_vectors_game_of_thrones-LIVE/master/data/got3.txt"
book4 = "https://raw.githubusercontent.com/llSourcell/word_vectors_game_of_thrones-LIVE/master/data/got4.txt"
book5 = "https://raw.githubusercontent.com/llSourcell/word_vectors_game_of_thrones-LIVE/master/data/got5.txt"

import requests
b1  = requests.get(book1)
b2  = requests.get(book2)
b3  = requests.get(book3)
b4  = requests.get(book4)
b5  = requests.get(book5)

book1_content = [sent for sent in b1.text.splitlines() if sent != '']
book2_content = [sent for sent in b2.text.splitlines() if sent != '']
book3_content = [sent for sent in b3.text.splitlines() if sent != '']
book4_content = [sent for sent in b4.text.splitlines() if sent != '']
book5_content = [sent for sent in b5.text.splitlines() if sent != '']

#all_content = book1_content+book2_content+book3_content+book4_content+book5_content

In [0]:
# clean the text
import re
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

# applying to all the books

book1_sentences = []
for raw_sentence in book1_content:
    if len(raw_sentence) > 0:
        book1_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

book2_sentences = []
for raw_sentence in book2_content:
    if len(raw_sentence) > 0:
        book2_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

book3_sentences = []
for raw_sentence in book3_content:
    if len(raw_sentence) > 0:
        book3_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

book4_sentences = []
for raw_sentence in book4_content:
    if len(raw_sentence) > 0:
        book4_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

book5_sentences = []
for raw_sentence in book5_content:
    if len(raw_sentence) > 0:
        book5_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

#all_book_sentences = []
#for raw_sentence in all_content:
#    if len(raw_sentence) > 0:
#        all_book_sentences.append(' '.join(sentence_to_wordlist(raw_sentence)))

In [0]:
# Encoding the text and extract embeddings in structured format
from bert_embedding import BertEmbedding
from tqdm import tqdm_notebook
import pandas as pd
import mxnet as mx
ctx = mx.gpu(0)

def generate_bert_embeddings(sentences):
  
  bert_embedding = BertEmbedding(ctx=ctx)
  print("Encoding Sentences:")
  result = bert_embedding(sentences)
  print('\n')
  print("Encoding Finished")
  

  df = pd.DataFrame()

  for i in tqdm_notebook(range(len(result))):
    embed = pd.DataFrame(result[i][1])
    embed['words'] = result[i][0]
    df = pd.concat([df, embed])
  
  return df


In [0]:
book1_embedding = generate_bert_embeddings(book1_sentences)
book2_embedding = generate_bert_embeddings(book2_sentences)
book3_embedding = generate_bert_embeddings(book3_sentences)
book4_embedding = generate_bert_embeddings(book4_sentences)
book5_embedding = generate_bert_embeddings(book5_sentences)
#all_book_embedding = generate_bert_embeddings(all_book_sentences)

Encoding Sentences:


Encoding Finished



Encoding Sentences:


Encoding Finished



Encoding Sentences:


Encoding Finished



Encoding Sentences:


Encoding Finished


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Encoding Sentences:


Encoding Finished


In [0]:
def reduce_dimension(embedding_df):
  from sklearn.decomposition import TruncatedSVD
  tsvd = TruncatedSVD(n_components=3)
  tsvd_3d = pd.DataFrame(tsvd.fit_transform(embedding_df.drop('words', axis=1)))
  tsvd_3d['words'] = embedding_df['words'].values

  from sklearn.decomposition import PCA
  pca = PCA(3)
  pca_3d = pd.DataFrame(pca.fit_transform(embedding_df.drop('words', axis=1)))
  pca_3d['words'] = embedding_df['words'].values
  return tsvd_3d, pca_3d


In [0]:
tsvd_book1, pca_book1 = reduce_dimension(book1_embedding)
tsvd_book2, pca_book2 = reduce_dimension(book2_embedding)
tsvd_book3, pca_book3 = reduce_dimension(book3_embedding)
tsvd_book4, pca_book4 = reduce_dimension(book4_embedding)
tsvd_book5, pca_book5 = reduce_dimension(book5_embedding)
#tsvd_all_book, pca_all_book = reduce_dimension(all_book_embedding)

In [0]:
tsvd_book1.to_csv("tsvd_book1.csv")
pca_book1.to_csv("pca_book1.csv")
tsvd_book2.to_csv("tsvd_book2.csv")
pca_book2.to_csv("pca_book2.csv")
tsvd_book3.to_csv("tsvd_book3.csv")
pca_book3.to_csv("pca_book3.csv")
tsvd_book4.to_csv("tsvd_book4.csv")
pca_book4.to_csv("pca_book4.csv")
tsvd_book5.to_csv("tsvd_book5.csv")
pca_book5.to_csv("pca_book5.csv")
#tsvd_all_book.to_csv("tsvd_all_book.csv")
#pca_all_book.to_csv("pca_all_book.csv")

In [0]:
from google.colab import files
for file in ["tsvd_book1.csv","pca_book1.csv","tsvd_book2.csv","pca_book2.csv","tsvd_book3.csv","pca_book3.csv","tsvd_book4.csv","pca_book4.csv","tsvd_book5.csv","pca_book5.csv"]:#,"tsvd_all_book.csv","pca_all_book.csv"]:
  files.download(file)